In [1]:
## load the df

import os, sys
import pandas as pd
from globes import taxi_dir, days_dir

filename = "zones_2016-01-20.csv"
filepath = os.path.join(taxi_dir, days_dir, filename)

df = pd.read_csv(filepath, parse_dates=['pickup_datetime', 'dropoff_datetime'], index_col=0)
print df

             pickup_datetime    dropoff_datetime  trip_distance  \
VendorID                                                          
2        2016-01-20 00:00:00 2016-01-20 00:15:03           3.47   
2        2016-01-20 00:00:00 2016-01-20 00:00:00          10.17   
2        2016-01-20 00:00:00 2016-01-20 00:00:00           3.89   
2        2016-01-20 00:00:00 2016-01-20 00:00:00          10.17   
2        2016-01-20 00:00:00 2016-01-20 00:00:00           1.43   
2        2016-01-20 00:00:00 2016-01-20 00:00:00           0.19   
2        2016-01-20 00:00:00 2016-01-20 00:00:00           0.19   
2        2016-01-20 00:00:00 2016-01-20 00:44:30           6.12   
2        2016-01-20 00:00:00 2016-01-20 00:00:00           6.34   
2        2016-01-20 00:00:00 2016-01-20 00:00:00           6.34   
2        2016-01-20 00:00:00 2016-01-20 00:00:00           8.72   
2        2016-01-20 00:00:00 2016-01-20 00:00:00           6.25   
2        2016-01-20 00:00:00 2016-01-20 00:00:00          12.1

In [2]:
import math

def getHalfHour(t):
    return "{:02d}:{:02d}".format(t.hour, int(math.floor(t.minute/30)))

## time delta groups
grouped = df.groupby(df['pickup_datetime'].map(getHalfHour))

In [7]:
import json
ZONE = "taxi"
ID = "LocationID"
NAME = "zone"

def getGeoJSON():
    zone_filename = "zones_" + ZONE + ".geojson"
    with open(zone_filename) as zone_file:
            zone_geojson = json.load(zone_file)
            zone_geojson = map(lambda x: x["properties"], zone_geojson["features"])
    zone_data = {}
    for zone in zone_geojson:
        zone_data[zone[ID]] = zone[NAME]
    return zone_data

zone_data = getGeoJSON()

## movement volume between every pair of zones
def getVolume(group):
    df = group.groupby(['pickup_zone_taxi', 'dropoff_zone_taxi'], as_index=False)
    return df['passenger_count'].agg(sum)

# for name, group in grouped:
#     vol = getVolume(group)
#     print vol

# overall; sort by total flow, transform ids to zone names
def totalFlow():
    grouped = getVolume(df)
    grouped = grouped.sort_values(by="passenger_count", ascending=False)
    grouped['pickup_zone_name'] = grouped['pickup_zone_taxi'].apply(lambda x: zone_data[x])
    grouped['dropoff_zone_name'] = grouped['dropoff_zone_taxi'].apply(lambda x: zone_data[x])
    return grouped

print totalFlow()


       pickup_zone_taxi  dropoff_zone_taxi  passenger_count  \
12094             237.0              236.0             3691   
11952             236.0              237.0             3263   
11951             236.0              236.0             3200   
12095             237.0              237.0             2714   
12052             237.0              162.0             1799   
98                  7.0                7.0             1789   
12346             239.0              238.0             1741   
11174             230.0              186.0             1737   
9541              186.0              161.0             1730   
12051             237.0              161.0             1692   
12216             238.0              239.0             1590   
9581              186.0              230.0             1578   
11734             234.0              170.0             1484   
8132              161.0              237.0             1455   
8267              162.0              170.0             